In [18]:
# ----- MAKE A COPY OF THIS -----

# Description: Uses UnsupervisedClassification on individual rasters
# Goal: Obtain 'actual' raster informations

In [19]:
# --- getting actual information  -----
def get_actual_table(project, net, date):
    outputTable = net + f"/{project}/Actual/Tables/Table{date}.csv"
    inputRaster = net + f"/{project}/Actual/Classified/ActualUnsuper{date}.tif"
    arcpy.ia.SummarizeCategoricalRaster(inputRaster, outputTable)

In [20]:
def get_actual_area(project, net, dates):
    result = {}
    for date in dates:
        path = network_path + f"/{project}/Actual/Tables/Table{date}.csv"
        df = pd.read_csv(path)
        pixelSum = df.sum(axis = 1).values[0]
        pixelFirst = df["C_1"].values[0] 
        percentageCovered = (pixelFirst/pixelSum) * 100
        if percentageCovered > 20: 
            # C_1 is unknown and big, take C3-C5
            pixelWater = df["C_3"].values[0] + df["C_4"].values[0] + df["C_5"].values[0] 
        elif percentageCovered > 8:
            # C_1 is unknown 
            pixelWater = df["C_2"].values[0] + df["C_3"].values[0]
        else:
            pixelWater = df["C_1"].values[0]

        percentageWater = (pixelWater/pixelSum) * 100
        percentageWater = round(percentageWater, 4)
        result[date] = percentageWater
    
    return result
# Notes:

# if percentage of C_1 is greater than 8%, that means C_1 is unknown
# if greater than 20, unknown is large
# 20220615 -- no unknown, C_1
# 20220625 -- no unknown, C_1
# 20220708 -- large unknown (>500k), C3 -> C_5
# 20220710 -- no unknown, C_1
# 20220720 -- no unknown, C_1
# 20220729 -- unknown (>100k), C_2 and C_3
# 20220813 -- unknown (>100k), C_2 and C_3
# 20220828 -- unknown (>100k), C_2 and C_3

In [21]:
def get_actual_bands(project, net, date, year): 
    try:
        #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/R60m/*B[01][1234]_60m.jp2"
        #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/R60m/*B[01][1234]_60m.jp2"
        fulldir = net + f"/Data/{year}/{date}/GRANULE/L*/IMG_DATA/R60m/*B[01][15]_60m.jp2"
        contents = glob.glob(fulldir)
        if not contents:
            #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/*B[01][1234].jp2"
            #fulldir = net + f"/Data/{date}/GRANULE/L*/IMG_DATA/*B[01][284].jp2"
            fulldir = net + f"/Data/{year}/{date}/GRANULE/L*/IMG_DATA/*B[01][15].jp2"
            
            contents = glob.glob(fulldir)
    except Exception as e:
        print("An error has occurred:", e)


    match = contents[1:len(contents)-1]
    #match = contents
    results = [e[len(e)-12:] for e in match]

    print(results)

    output = f"/{project}/Actual/Unclipped/CompositeBand{date}.tif"
    output = net + output

    
    arcpy.management.CompositeBands(match, output)
    return output

In [22]:
def clip_actual_raster(project, net, img, date): 
    clip_shape = net + "/Unsupervised/ExpandedShape/ExpandedShape.shp"
    clipped_destination = net + f"/{project}/Actual/Clipped/ActualRaster{date}.tif"
    arcpy.management.Clip(img, clip_shape, clipped_destination)
    return clipped_destination

In [23]:
# --- train on all datasets ---#
# to find errors,

def train_actual(project, net, img, date):  
    classes = 12
    class_size = 5
    samp_interval = 4
    sig_file = net + f"/{project}/Actual/Signature/sigfile{date}.gsg"
    out_unsuper = IsoClusterUnsupervisedClassification(img, classes, class_size, samp_interval, sig_file)
    out_unsuper.save(net + f"/{project}/Actual/Classified/ActualUnsuper{date}.tif")
    print("Done training")
    return out_unsuper



In [44]:
def create_table(net):
    df = pd.DataFrame(columns = ["Year", "X", "Y", "Type", "Bands"])
    directory = net + "/CSV/main.csv"
    if os.path.exists(directory):
        return
    else:
        print("Creating a new table...")
        df.to_csv(directory, index = False)
    

In [48]:
def save_actual_table(net, obj, year, method, bands):
    create_table(net)
    new_obj = {"X": obj.keys(),
               "Y": obj.values(),
               "Year": year,
               "Type": method,
                "Bands": bands}
    df = pd.DataFrame(new_obj)
    temp = df.to_csv(net + "/CSV/temp.csv", index = False)
    
    print("Appending new information...")
    existing_df = pd.read_csv(net + "/CSV/main.csv")
    new_df = pd.read_csv(net + "/CSV/temp.csv")
  
    existing_df = pd.concat([existing_df, new_df], ignore_index = True)
    existing_df = existing_df.drop_duplicates()

    existing_df.to_csv(net + "/CSV/main.csv", index = False)



In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import arcpy
import os
import glob
from arcpy import env
from arcpy.sa import *
from arcpy.ia import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

arcpy.env.overwriteOutput = True
network_path = "network_path" 
arcpy.CheckOutExtension("ImageAnalyst")

try:
    contents = os.listdir(network_path)
except FileNotFoundError:
    print("NAN")
except Exception as e:
    print("An error has occurred:", e)

    
# ------ CHANGE THESE VARIABLES -----
proj = "2021"
year = "2021"
dates = glob.glob(network_path + f"/Data/{year}/*")
dates = [d[len(d)-8:] for d in dates]
method = "Actual"
bands = "5,"


In [34]:
# ----- Getting and Clipping Bands -----
for date in dates:
    out = get_actual_bands(proj, network_path, date, year)
    out_clipped = clip_actual_raster(proj, network_path, out, date)
print("Clipped all Actual Rasters")


['0911_B05.jp2']
['1809_B05.jp2']
['0911_B05.jp2']
['0911_B05.jp2']
['_B05_60m.jp2']
['1911_B05.jp2']
['1809_B05.jp2']
['1809_B05.jp2']
['0911_B05.jp2']
Clipped all Actual Rasters


In [35]:
# ----- Apply Unsupervised Training on all Raster -----
for date in dates:
    in_raster = network_path + f"/{proj}/Actual/Clipped/ActualRaster{date}.tif"
    result = train_actual(proj, network_path, in_raster, date)

Done training
Done training
Done training
Done training
Done training
Done training
Done training
Done training
Done training


In [36]:
# ----- Get Pixel information -----
for date in dates:
    get_actual_table(proj, network_path, date)

In [49]:
# ----- Get Area, upload as new csv file -----
obj = get_actual_area(proj, network_path, dates)
method = "Actual"
save_actual_table(network_path, obj, year, method, bands)
print("Saved")

Appending new information...
Saved


In [ ]:
# --------------------------------------------------------- END -------------------------------------------------